<a href="https://colab.research.google.com/github/chlgudwnss/Ndvia_yolo/blob/20250404/yolo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#필수 라이브러리 설치
!apt-get install -y libgl1-mesa-glx
!pip install -U yt-dlp ultralytics opencv-python

import yt_dlp
import cv2
import time
import os
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

#YouTube 영상 다운로드 (yt-dlp 사용)
def download_youtube_video(url, output_path="traffic.mp4"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best',
        'outtmpl': output_path,  # 저장할 파일 경로
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"✅ YouTube 영상 다운로드 완료: {output_path}")
    except Exception as e:
        print(f"❌ YouTube 다운로드 오류: {e}")

#**새로운 YouTube 영상 URL 적용**
video_url = 'https://www.youtube.com/shorts/NJ4kfYqbvEQ'  # 새로운 영상 URL
download_youtube_video(video_url, "/content/traffic.mp4")

#YOLOv11x 모델 로드
model = YOLO('yolo11x.pt')  #변경된 모델명

#차량 감지 및 카운팅
cap = cv2.VideoCapture("/content/traffic.mp4")

if not cap.isOpened():
    print("❌ 비디오 파일을 열 수 없습니다. 다운로드가 실패했을 가능성이 있습니다.")
else:
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)  # FPS 정보 가져오기
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 총 프레임 수
    duration = total_frames / fps  # 전체 영상 길이(초)

    print(f"🎥 영상 정보: {duration:.2f}초, {fps:.2f} FPS, 총 {total_frames} 프레임")

    # ✅ MP4V 코덱 사용
    out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), int(fps), (frame_width, frame_height))

    # VideoWriter 정상 작동 여부 확인
    if not out.isOpened():
        print("❌ VideoWriter가 정상적으로 열리지 않았습니다. 코덱을 확인하세요.")
    else:
        print("✅ VideoWriter 초기화 성공")

    car_ids = set()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 🚗 차량 감지 및 추적
        results = model.track(frame, persist=True, tracker="bytetrack.yaml", classes=[2,3,5,7], conf=0.5)

        if results and results[0].boxes.id is not None:
            for box_id in results[0].boxes.id.cpu().numpy().astype(int):
                car_ids.add(box_id)

        # 감지된 프레임 저장
        annotated_frame = results[0].plot() if results else frame
        out.write(annotated_frame)

    cap.release()
    out.release()

    #결과 출력
    print(f"🔹 전체 영상에서 통과한 차량 수: {len(car_ids)}대")
    print("\n📌 결과 영상 확인 방법:")
    print("1. 좌측 폴더 아이콘 📁 클릭")
    print("2. `/content/output.mp4` 파일 우클릭 → [다운로드] 또는 [미리보기]")

    #output.mp4 파일 존재 여부 확인 후 다운로드
    output_path = "/content/output.mp4"
    if os.path.exists(output_path):
        print("✅ output.mp4 파일이 정상적으로 저장되었습니다.")
        from google.colab import files
        files.download(output_path)
    else:
        print("❌ output.mp4 파일이 생성되지 않았습니다. OpenCV 설정을 확인하세요.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
  Using cached yt_dlp-2025.3.31-py3-none-any.whl.metadata (172 kB)
  Using cached ultralytics-8.3.101-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86

100%|██████████| 109M/109M [00:00<00:00, 259MB/s]


🎥 영상 정보: 6.38초, 23.97 FPS, 총 153 프레임
✅ VideoWriter 초기화 성공
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.5s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 640x384 5 cars, 1 truck, 102.2ms
Speed: 19.4ms preprocess, 102.2ms inference, 463.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 truck, 57.5ms
Speed: 3.2ms preprocess, 57.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 truck, 57.6ms
Speed: 4.1ms preprocess, 57.6ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 truck, 50.5ms
Speed: 3.4ms preprocess, 50.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 1 truck, 50.5ms
Speed: 2.8ms preprocess, 50.5ms inference, 2.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>